In [ ]:
%%capture
!pip install datasets transformers
!apt install git-lfs

In [ ]:
import huggingface_hub
from huggingface_hub import notebook_login
# Enter yours
your_token = '' 
huggingface_hub.login(token = your_token)

In [ ]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]

In [ ]:
task = "rte"
model_checkpoint = "bert-base-uncased"
batch_size = 64

In [ ]:
#from datasets import load_dataset, load_metric

from datasets import load_dataset
!pip install evaluate
from evaluate import load

In [ ]:
actual_task = "mnli" if task == "mnli-mm" else task
print(f"Task is: {actual_task} ")
dataset = load_dataset("glue", actual_task)

!pip install scikit-learn
metric = load("accuracy")

Task is: rte 


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 2490
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 277
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3000
    })
})

To access an actual element, you need to select a split first, then give an index:

In [ ]:
dataset["train"][0]

{'sentence1': 'No Weapons of Mass Destruction Found in Iraq Yet.',
 'sentence2': 'Weapons of Mass Destruction Found in Iraq.',
 'label': 1,
 'idx': 0}

In [ ]:
dataset["train"].to_pandas()

,sentence1,sentence2,label,idx
0,No Weapons of Mass Destruction Found in Iraq Yet.,Weapons of Mass Destruction Found in Iraq.,1,0
1,"A place of sorrow, after Pope John Paul II die...",Pope Benedict XVI is the new leader of the Rom...,0,1
2,Herceptin was already approved to treat the si...,Herceptin can be used to treat breast cancer.,0,2
3,"Judie Vivian, chief executive at ProMedica, a ...",The previous name of Ho Chi Minh City was Saigon.,0,3
4,A man is due in court later charged with the m...,Paul Stewart Hutchinson is accused of having s...,1,4
...,...,...,...,...
2485,There is none. They found as many weapons in t...,Weapons of mass destruction found in Iraq.,1,2485
2486,"Dr. Eric Goosby, a pioneer in the fight agains...",Pepfar is committed to fighting AIDS.,0,2486
2487,"NASA's Saturn exploration spacecraft, Cassini ...",Titan is the fifteenth of Saturn's known satel...,1,2487
2488,Brooklyn Borough Hall featured a Who's Who in ...,The Brooklyn Book Festival is held in Brooklyn...,0,2488


In [ ]:
dataset["validation"].to_pandas()

,sentence1,sentence2,label,idx
0,"Dana Reeve, the widow of the actor Christopher...",Christopher Reeve had an accident.,1,0
1,"Yet, we now are discovering that antibiotics a...",Bacteria is winning the war against antibiotics.,0,1
2,Cairo is now home to some 15 million people - ...,15 million tonnes of rubbish are produced dail...,1,2
3,"The Amish community in Pennsylvania, which num...",Pennsylvania has the biggest Amish community i...,1,3
4,Security forces were on high alert after an el...,Security forces were on high alert after a cam...,0,4
...,...,...,...,...
272,Swansea striker Lee Trundle has negotiated a l...,Lee Trundle is in business with the League One...,0,272
273,A light blue 1975 Ford Escort GL once owned by...,A Houston multimillionaire buys the Pope's For...,0,273
274,"On October 1 2001, EU and other countries intr...","In 2001, the EU introduced a passport for pets.",0,274
275,Kozlowski and the company's former chief finan...,"Kozlowski was sentenced, Monday, to serve up t...",0,275


In [ ]:
dataset['test'].to_pandas()

,sentence1,sentence2,label,idx
0,Mangla was summoned after Madhumita's sister N...,Shukla is related to Mangla.,-1,0
1,Authorities in Brazil say that more than 200 p...,Authorities in Brazil hold 200 people as hostage.,-1,1
2,A mercenary group faithful to the warmongering...,An interior ministry worker was killed by a me...,-1,2
3,"The British ambassador to Egypt, Derek Plumbly...",Derek Plumbly resides in Egypt.,-1,3
4,Tibone estimated diamond production at four mi...,Botswana is a business partner of De Beers.,-1,4
...,...,...,...,...
2995,A small team from Britain's Scotland Yard has ...,Benazir Bhutto is dead.,-1,2995
2996,"The current Iraqi president, Jalal Talabani, s...",Jalal Talabani agrees to divide Iraq.,-1,2996
2997,"J K Rowling, the author of the ""Harry Potter"" ...","J.K. Rowling is the author of ""Harry Potter"".",-1,2997
2998,The forest was estimated to contain trees that...,Fossilized trees have been found in the USA.,-1,2998


In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(dataset["train"])

,sentence1,sentence2,label,idx
0,"Councilman Pierce said he was advised that the car belonged to James Clark, 68, an acquaintance of the Jones family.",Councilman Pierce owns a car.,not_entailment,1391
1,"Spain will recruit 500 extra police to fight Basque separatists ETA and Islamist militants in the next four years, Interior Minister Alfredo Perez Rubalcaba said on Tuesday. Including the new recruits, the number of anti-terrorism officers will have increased by 1,800 since the Socialist government took office in 2004, days after Islamist militants killed 191 people by bombing Madrid trains, Rubalcaba told a parliamentary committee. ETA has killed more than 800 people in four decades in its fight for an independent Basque Country composed of part of northern Spain and southern France. Its last fatal attack came on May 14, authorities believe, when a car bomb wrecked a police barracks in the Spanish Basque Country, killing one officer. The government will also provide an additional 800 officers to combat organised crime and another 1,248 staff to tackle domestic violence, Rubalcaba said.",ETA fights for the independence of Basque Country.,entailment,1091
2,"A computer system failure closed down share trading at the Tokyo Stock Exchange for most of yesterday, the worst disruption to date for Asia's largest bourse.",The Tokyo Stock Exchange was closed down by computer system failure.,entailment,456
3,U.S. President George W. Bush and Congressional leaders from both political parties have reached agreement on US$150 billion in tax relief in an effort to help stimulate the United States economy. VOA White House Correspondent Scott Stearns has the story. The agreement includes provision to give up to US$600 in tax rebates to millions of wage earners and an additional US$300 refund for each child. It allows businesses to immediately write off 50 percent of capital equipment purchases.,The Congress wants to boost the economy.,entailment,169
4,"On April 27th, Pavolini went on, ahead of his troops, and reached Mussolini. Accompanying him were Angela Curti, the former mistress of Mussolini.",Angela Curti was Mussolini's mistress.,entailment,905
5,"Alex Dyer, spokesman for the group, stated that Santarchy in Auckland is part of a worldwide phenomenon.",Alex Dyer represents Santarchy.,not_entailment,1693
6,"The cholera epidemic in Zimbabwe appears to have passed its peak, says the World Health Organization (WHO). The number of new cases recorded in the week to mid-March nearly halved to 2,000, against 3,800 the preceding week and 8,000 cases a week in February. But the agency warned the weekly statistics were not always accurate. There have been more than 90,000 cholera cases in Zimbabwe since the start of the epidemic last August, about 4,000 of them fatal.","Since last August about 4,000 people died of cholera in Zimbabwe.",entailment,1381
7,"San Salvador, Jan. 13, '90 (Acan-Efe) -The bodies of Hector Oqueli and Gilda Flores, who had been kidnapped yesterday, were found in Cuilapa, Guatemala, near the border with El Salvador, the relatives of one of the victims have reported.",Guatemala borders on El Salvador.,entailment,432
8,Two women from two counties are dead and police have some clues as to who murdered them.,Police gather clues in double assassination.,entailment,843
9,Thousands of Romanians participated in rallies on Monday to campaign for the release of three Romanian journalists kidnapped last month in Iraq . Crowds swelled in size after a weekend of smaller rallies sponsored by Romanian media workers and the Arab community in Romania.,Romania sent 730 troops in Iraq.,not_entailment,458


In [ ]:
import numpy as np

fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
metric.compute(predictions=fake_preds, references=fake_labels)

{'accuracy': 0.484375}

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [ ]:
tokenizer("Hello, this one sentence!", max_length=50, padding="max_length", truncation=True)

{'input_ids': [101, 7592, 1010, 2023, 2028, 6251, 999, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [ ]:
tokenizer("Hello, this one sentence!")

{'input_ids': [101, 7592, 1010, 2023, 2028, 6251, 999, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer("Hello, this one sentence!","What is going on here!")

{'input_ids': [101, 7592, 1010, 2023, 2028, 6251, 999, 102, 2054, 2003, 2183, 2006, 2182, 999, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

In [ ]:
dataset['train'][0]

{'sentence1': 'No Weapons of Mass Destruction Found in Iraq Yet.',
 'sentence2': 'Weapons of Mass Destruction Found in Iraq.',
 'label': 1,
 'idx': 0}

In [ ]:
sentence1_key, sentence2_key = task_to_keys[task]
if sentence2_key is None:
    print(f"Sentence: {dataset['train'][0][sentence1_key]}")
else:
    print(f"Sentence 1: {dataset['train'][0][sentence1_key]}")
    print(f"Sentence 2: {dataset['train'][0][sentence2_key]}")

Sentence 1: No Weapons of Mass Destruction Found in Iraq Yet.
Sentence 2: Weapons of Mass Destruction Found in Iraq.


In [ ]:
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)

In [ ]:
preprocess_function(dataset['train'][:1])


{'input_ids': [[101, 2053, 4255, 1997, 3742, 6215, 2179, 1999, 5712, 2664, 1012, 102, 4255, 1997, 3742, 6215, 2179, 1999, 5712, 1012, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [ ]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

In [ ]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2490
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 277
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3000
    })
})

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
from transformers import logging

logging.set_verbosity_error() #Since we learn the meaning of warning, we can silence it.

num_labels = 3 if task.startswith("mnli") else 1 if task=="stsb" else 2

model_checkpoint = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)
model.dropout.p = 0.2


In [ ]:
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]
run_no="3"
args = TrainingArguments(
    f"{model_name}-finetuned-{task}-run_{run_no}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=2,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

<ipython-input-30-4d7a7b54288c>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Recall that we have CUDA device. Lets check the amount of memory.
!nvidia-smi

Mon Apr  7 10:59:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P0             49W /  400W |     895MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: bilkentbitirmehavelsan (bilkentbitirmehavelsan-bilkent-niversitesi) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


{'eval_loss': 0.6800429224967957, 'eval_accuracy': 0.6028880866425993, 'eval_runtime': 0.6639, 'eval_samples_per_second': 417.212, 'eval_steps_per_second': 7.531, 'epoch': 1.0}
{'eval_loss': 0.6681187748908997, 'eval_accuracy': 0.6245487364620939, 'eval_runtime': 0.6556, 'eval_samples_per_second': 422.541, 'eval_steps_per_second': 7.627, 'epoch': 2.0}
{'train_runtime': 44.6039, 'train_samples_per_second': 111.649, 'train_steps_per_second': 1.749, 'train_loss': 0.6901067097981771, 'epoch': 2.0}


TrainOutput(global_step=78, training_loss=0.6901067097981771, metrics={'train_runtime': 44.6039, 'train_samples_per_second': 111.649, 'train_steps_per_second': 1.749, 'train_loss': 0.6901067097981771, 'epoch': 2.0})

We can check with the `evaluate` method that our `Trainer` did reload the best model properly (if it was not the last one):

In [ ]:
trainer.evaluate()

{'eval_loss': 0.6681187748908997, 'eval_accuracy': 0.6245487364620939, 'eval_runtime': 0.6847, 'eval_samples_per_second': 404.546, 'eval_steps_per_second': 7.302, 'epoch': 2.0}


{'eval_loss': 0.6681187748908997,
 'eval_accuracy': 0.6245487364620939,
 'eval_runtime': 0.6847,
 'eval_samples_per_second': 404.546,
 'eval_steps_per_second': 7.302,
 'epoch': 2.0}

In [ ]:
%%capture
trainer.push_to_hub()

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("berkayaltntas/bert-base-uncased-finetuned-rte-run_3")
#check your own model also
print(model)

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
%%capture
! pip install optuna
# ! pip install ray[tune]

In [ ]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)



In [ ]:
trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

<ipython-input-38-900d107e0d38>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
import optuna

def my_hp_space(trial: optuna.trial.Trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 5e-5, log=True),
        "num_train_epochs": trial.suggest_categorical("num_train_epochs", [2, 3, 4]),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [16, 32]),
        "max_length": trial.suggest_categorical("max_length", [128, 256, 512]),
        "classifier_dropout": trial.suggest_float("classifier_dropout", 0.1, 0.5),
    }

In [ ]:
best_run = trainer.hyperparameter_search(hp_space = my_hp_space, backend = "optuna", n_trials=24, direction="maximize")

[I 2025-04-07 11:01:39,881] A new study created in memory with name: no-name-fa7979f2-e3c1-4a85-bde4-8956ca4948ab


eval/accuracy,▁██
eval/loss,█▁▁
eval/runtime,▃▁█
eval/samples_per_second,▆█▁
eval/steps_per_second,▆█▁
train/epoch,▁███
train/global_step,▁███
eval/accuracy,0.62455
eval/loss,0.66812
eval/runtime,0.6847
eval/samples_per_second,404.546


{'eval_loss': 0.7057711482048035, 'eval_accuracy': 0.49097472924187724, 'eval_runtime': 0.6548, 'eval_samples_per_second': 423.062, 'eval_steps_per_second': 7.636, 'epoch': 1.0}
{'eval_loss': 0.6515233516693115, 'eval_accuracy': 0.631768953068592, 'eval_runtime': 0.6593, 'eval_samples_per_second': 420.154, 'eval_steps_per_second': 7.584, 'epoch': 2.0}
{'eval_loss': 0.6529138684272766, 'eval_accuracy': 0.6425992779783394, 'eval_runtime': 0.6631, 'eval_samples_per_second': 417.743, 'eval_steps_per_second': 7.54, 'epoch': 3.0}
{'train_runtime': 62.7448, 'train_samples_per_second': 119.054, 'train_steps_per_second': 3.729, 'train_loss': 0.6316716120793269, 'epoch': 3.0}


[I 2025-04-07 11:03:05,619] Trial 0 finished with value: 0.6425992779783394 and parameters: {'learning_rate': 1.632929904930613e-05, 'num_train_epochs': 3, 'per_device_train_batch_size': 32, 'max_length': 512, 'classifier_dropout': 0.24469588923063107}. Best is trial 0 with value: 0.6425992779783394.


eval/accuracy,▁▇█
eval/loss,█▁▁
eval/runtime,▁▅█
eval/samples_per_second,█▄▁
eval/steps_per_second,█▄▁
train/epoch,▁▅██
train/global_step,▁▅██
eval/accuracy,0.6426
eval/loss,0.65291
eval/runtime,0.6631
eval/samples_per_second,417.743


...

[I 2025-04-07 11:36:17,960] Trial 22 pruned. 


{'eval_loss': 0.6643483638763428, 'eval_accuracy': 0.6028880866425993, 'eval_runtime': 0.6551, 'eval_samples_per_second': 422.826, 'eval_steps_per_second': 7.632, 'epoch': 1.0}


eval/accuracy,▁
eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁
train/global_step,▁
eval/accuracy,0.60289
eval/loss,0.66435
eval/runtime,0.6551
eval/samples_per_second,422.826


[I 2025-04-07 11:36:37,714] Trial 23 pruned. 


{'eval_loss': 0.7011489868164062, 'eval_accuracy': 0.4981949458483754, 'eval_runtime': 0.6577, 'eval_samples_per_second': 421.19, 'eval_steps_per_second': 7.603, 'epoch': 1.0}


In [ ]:
best_run

BestRun(run_id='4', objective=0.6823104693140795, hyperparameters={'learning_rate': 3.7693294228753754e-05, 'num_train_epochs': 4, 'per_device_train_batch_size': 16, 'max_length': 512, 'classifier_dropout': 0.10166181013063974}, run_summary=None)

In [ ]:
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

trainer.train()


{'eval_loss': 0.6470687389373779, 'eval_accuracy': 0.628158844765343, 'eval_runtime': 0.6546, 'eval_samples_per_second': 423.142, 'eval_steps_per_second': 7.638, 'epoch': 1.0}


It seems you are trying to upload a large folder at once. This might take some time and then fail if the folder is too large. For such cases, it is recommended to upload in smaller batches or to use `HfApi().upload_large_folder(...)`/`huggingface-cli upload-large-folder` instead. For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/upload#upload-a-large-folder.


{'eval_loss': 0.6231392025947571, 'eval_accuracy': 0.6750902527075813, 'eval_runtime': 0.6552, 'eval_samples_per_second': 422.76, 'eval_steps_per_second': 7.631, 'epoch': 2.0}
{'eval_loss': 0.8376210331916809, 'eval_accuracy': 0.6714801444043321, 'eval_runtime': 0.657, 'eval_samples_per_second': 421.598, 'eval_steps_per_second': 7.61, 'epoch': 3.0}
{'loss': 0.4742, 'grad_norm': 13.06845760345459, 'learning_rate': 7.4903341095600405e-06, 'epoch': 3.2051282051282053}
{'eval_loss': 1.058768391609192, 'eval_accuracy': 0.6750902527075813, 'eval_runtime': 0.6548, 'eval_samples_per_second': 423.036, 'eval_steps_per_second': 7.636, 'epoch': 4.0}
{'train_runtime': 79.3126, 'train_samples_per_second': 125.579, 'train_steps_per_second': 7.868, 'train_loss': 0.4113004207611084, 'epoch': 4.0}


TrainOutput(global_step=624, training_loss=0.4113004207611084, metrics={'train_runtime': 79.3126, 'train_samples_per_second': 125.579, 'train_steps_per_second': 7.868, 'train_loss': 0.4113004207611084, 'epoch': 4.0})

In [ ]:
trainer.push_to_hub()

It seems you are trying to upload a large folder at once. This might take some time and then fail if the folder is too large. For such cases, it is recommended to upload in smaller batches or to use `HfApi().upload_large_folder(...)`/`huggingface-cli upload-large-folder` instead. For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/upload#upload-a-large-folder.


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/berkayaltntas/bert-base-uncased-finetuned-rte-run_3/commit/3190af991288a0de5ba6d2f8a6775d260cd24575', commit_message='End of training', commit_description='', oid='3190af991288a0de5ba6d2f8a6775d260cd24575', pr_url=None, repo_url=RepoUrl('https://huggingface.co/berkayaltntas/bert-base-uncased-finetuned-rte-run_3', endpoint='https://huggingface.co', repo_type='model', repo_id='berkayaltntas/bert-base-uncased-finetuned-rte-run_3'), pr_revision=None, pr_num=None)